Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# PyTorch Pretrained BERT on AzureML with SQuAD Dataset
**BERT: Re-training of Deep Bidirectional Transformers for Language Understanding** [\[1\]](#References)

## What is BERT?

[BERT (Bidirectional Encoder Representations from Transformers)](https://arxiv.org/abs/1810.04805) is a powerful pre-trained language model by presenting state-of-the-art results in a wide variety of NLP tasks, including Question Answering (SQuAD v1.1), Natural language Inference (MNLI), Text Classification, Name Entity Recognition, etc., by only a few epochs of fine tuning on task specific datasets. The key technical innovation of BERT is applying the bidirectional training of Transformer, which is a popular attention model that learns contextual relations between words (or sub-words) in a text.

## How to fine-tuning BERT for QA?

The figure below shows how BERT can be fine tuned for Question and Answering (QA) task. BERT plugs the question-passage pairs in SQuAD dataset as the inputs, and the `[SEP]` representation is a special separator token for separating questions/answers. At the output layer, it outputs `Start/End` to denote the answer in the paragraph.

<img src="https://nlpbp.blob.core.windows.net/images/bertqa.PNG" height=400 width=400>

## What is SQuAD dataset?

"Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable". [\[2\]](#References)

"SQuAD 1.1, the previous version of the SQuAD dataset, contains 100,000+ question-answer pairs on 500+ articles". [\[2\]](#References) More details from [https://rajpurkar.github.io/SQuAD-explorer/](https://rajpurkar.github.io/SQuAD-explorer/).

This notebook contains an end-to-end walkthrough of using Azure Machine Learning Service to run [PyTorch reimplementation](https://github.com/huggingface/pytorch-pretrained-BERT) of [Google's TensorFlow repository for the BERT model](https://github.com/google-research/bert) developed by Hugging Face.This notebook refers [Azure-BERT](https://github.com/microsoft/AzureML-BERT) created by Microsoft. We revised the notebook by updating deprecated Azureml utilites, adding more explanations and inducing more experiments on various AzureML GPU clusters.

You will find the following contents:
- Download SQuAD dataset on the remote compute and store them in Azure storage
- Fine-tuning BERT with distributed PyTorch by Horovod for SQuAD dataset on AzureML GPU clusters
- Further fine-tune BERT wtih AzureML hyperparameter optimizer 

# Table of Contents
0. [Prerequisites: Global Settings](#0.-Prerequisites:-Global-Settings)
1. [Data Loading](#1.-Data-Loading)    
    * 1.1. [Default AzureML datastore](#1.1-Default-AzureML-datastore)  
    * 1.2. [Download training dataset - SQuAD](#1.2-Download-training-dataset---SQuAD)  
    * 1.3. [Upload to Azure blob storage](#1.3-Upload-to-Azure-blob-storage)  
2. [Fine tuning BERT with Distributed Training by Horovod](#2.-Fine-tuning-BERT-with-Distributed-Training-by-Horovod)  
    * 2.1 [Create or Attach Existing AmlCompute](#2.1-Create-a-GPU-remote-compute-target)  
    * 2.2. [Access to a Project Directory](#2.2-Create-a-project-directory)  
    * 2.3. [Train Model on the Remote Compute](#2.3-Prepare-your-training-script)  
    * 2.4. [Create a PyTorch estimator for fine tuning](#2.4-Create-a-PyTorch-estimator-for-fine-tuning)  
    * 2.5. [Create an experiment](#2.5-Create-an-experiment)  
    * 2.6. [Submit and Monitor your run](#2.6-Submit-and-Monitor-your-run)  
3. [Fine Tuning BERT with Hyperparameter Tuning](#3-Fine-Tuning-BERT-with-Hyperparameter-Tuning)
    * 3.1 [Start a hyperparameter sweep](#3.1-Start-a-hyperparameter-sweep)
    * 3.2 [Monitor HyperDrive runs](#3.2-Monitor-HyperDrive-runs)
    * 3.3 [Find and register the best model](#3.3-Find-and-register-the-best-model)
- [References](#References)

## 0. Prerequisites: Global Settings
- Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)

- Go through the [Configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`.

- Install the Python SDK:  make sure to install notebook, and contrib
```
conda create -n azureml -y Python=3.6
source activate azureml
pip install --upgrade azureml-sdk[notebooks,contrib] 
conda install ipywidgets
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable azureml.widgets --user --py
```

You will need to restart jupyter after this
Detailed instructions are here: https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python 

Run the following cell to make sure you have installed all the packages.

In [1]:
import sys
sys.path.append("../../")
import urllib, os
from utils_nlp.azureml import azureml_utils
import math
import json 
import pandas as pd
#package for flattening json in pandas df
from pandas.io.json import json_normalize
import shutil
# Check core SDK version number
import azureml.core
from azureml.core import Datastore
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import MpiConfiguration
from azureml.telemetry import set_diagnostics_collection
from azureml.train.dnn import PyTorch
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.48


**Initialize workspace**

To create or access an Azure ML Workspace, you will need to import the AML library and the following information:
* A name for your workspace
* Your subscription id
* The resource group name

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the second Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [2]:
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>"
)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.
Workspace name: MAIDAIPBERT-eastus
Azure region: eastus
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: nlprg


Opt-in diagnostics for better experience, quality, and security of future releases.

In [3]:
set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## 1. Data Loading

In this section, we will
1. Get default AzureML datastore
2. Download the dataset and load the dataset
3. Upload the training set to the default blob storage of the workspace

In [4]:
data_folder = './squad'

### 1.1 Default AzureML datastore

To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets.

Each workspace is associated with a default Azure Blob datastore named `'workspaceblobstore'`. In this work, we use this default datastore to collect the SQuAD training data .

In [5]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name, ds.as_mount())

AzureBlob maidaipberteas6144514557 azureml-blobstore-cf97de17-8d21-437f-8b4c-298560f34ecd $AZUREML_DATAREFERENCE_workspaceblobstore


### 1.2 Download training dataset - SQuAD

SQuAD dataset can be downloaded with the following links and should be saved in a blob storage.
- [train-v1.1.json](https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json)
- [dev-v1.1.json](https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json)

In [14]:
os.makedirs('./squad', exist_ok=True)
urllib.request.urlretrieve('https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json', filename=os.path.join(data_folder, 'train-v1.1.json'))
urllib.request.urlretrieve('https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json', filename=os.path.join(data_folder, 'dev-v1.1.json'))

('./squad\\dev-v1.1.json', <http.client.HTTPMessage at 0x1e1a85ef198>)

The SQuAD dataset contains question-answer pairs on 500+ articles. For each observation in the training set, we have a **context, question, and text**. An example shown as below: [source](https://towardsdatascience.com/building-a-question-answering-system-part-1-9388aadff507)
<img src="https://nlpbp.blob.core.windows.net/images/squad.png">

In [57]:
#load json object
with open(os.path.join(data_folder, 'train-v1.1.json')) as train_file:
    train = json.load(train_file)
for paragraph in train['data'][0]['paragraphs']:
    answer_question = paragraph['qas']
    context = paragraph['context']
    paragraph = paragraph
    break
print("The structure of an example in the training data.")
json_normalize(paragraph).head()

The structure of an example in the training data.


,context,qas
0,"Architecturally, the school has a Catholic cha...","[{'answers': [{'answer_start': 515, 'text': 'S..."


In [58]:
print("The structure of question answer pairs in the above example.")
json_normalize(answer_question).head()

The structure of question answer pairs in the above example.


,answers,id,question
0,"[{'answer_start': 515, 'text': 'Saint Bernadet...",5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...
1,"[{'answer_start': 188, 'text': 'a copper statu...",5733be284776f4190066117f,What is in front of the Notre Dame Main Building?
2,"[{'answer_start': 279, 'text': 'the Main Build...",5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...
3,"[{'answer_start': 381, 'text': 'a Marian place...",5733be284776f41900661181,What is the Grotto at Notre Dame?
4,"[{'answer_start': 92, 'text': 'a golden statue...",5733be284776f4190066117e,What sits on top of the Main Building at Notre...


### 1.3 Upload to Azure blob storage

The following code will upload the SQuAD dataset to the path ./squad on the default datastore.

In [15]:
ds.upload(src_dir='./squad', target_path='./squad')

WARNING - Target already exists. Skipping upload for ./squad\dev-v1.1.json
WARNING - Target already exists. Skipping upload for ./squad\train-v1.1.json


$AZUREML_DATAREFERENCE_5a4cead96ec140b8b5884e917df16e3a

## 2. Fine-tuning BERT with Distributed Training by Horovod
As our `SQuAD` dataset are ready in Azure storage, we can start the fine-tune the model by exploting the power of distributed training on AzureML GPU clusters.

Once you've created your workspace and set up your development environment, training a model in Azure Machine Learning involves the following steps:
1. Create a GPU remote compute target
2. Create a project directory
3. Prepare your training script
4. Create an Estimator object
5. Submit the estimator to an experiment object under the workspace

### 2.1 Create a GPU remote compute target

We need to create a GPU [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to perform the fine-tuning. In this example, we create an AmlCompute cluster as our training compute resource. Please find the information of Azure VM size in below table.

    
|    VM Size    	| CPU 	|   GPU   	| Storage (SSD) 	| GPU memory 	| InfiniBand  	|
|:-------------:	|:---:	|:-------:	|:-------------:	|:----------:	|:----------:	|
|  Standard_NC6 	|  6  	| 1 x K80 	|    340 GiB    	|    8 GiB   	|      No   	|
| Standard_NC12 	|  12 	| 2 x K80 	|    680 GiB    	|   16 GiB   	|      No   	|
| Standard_NC24 	|  24 	| 4 x K80 	|    1440 GiB   	|   32 GiB   	|      No   	|
| Standard_NC24r 	|  24 	| 4 x K80 	|    1440 GiB   	|   32 GiB   	|      Yes   	|
| Standard_NC6s_v3 	|  6  	| 1 x V100 	|    736 GiB    	|   16 GiB   	|      No   	|
| Standard_NC12s_v3 |  12 	| 2 x V100 	|    1474 GiB   	|   32 GiB   	|      No   	|
| Standard_NC24s_v3 |  24 	| 4 x V100 	|    2948 GiB   	|   64 GiB   	|      No   	|
| Standard_NC24rs_v3|  24 	| 4 x V100 	|    2948 GiB   	|   64 GiB   	|      Yes   	|

This code creates a cluster for you if it does not already exist in your workspace.

***We strongly recommend to use NCv3-series (NVIDIA Tesla V100) to fine-tune with SQuAD dataset. You will need to request quota of NCv3-series for your AzureML subscription.***

In [6]:
# choose a name for your cluster
cluster_name = "bertncrs24"

try:
    gpu_compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24rs_v3',
                                                           max_nodes=4)

    # create the cluster
    gpu_compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    gpu_compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(gpu_compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-07-12T22:32:24.801000+00:00', 'errors': None, 'creationTime': '2019-07-12T19:59:45.933132+00:00', 'modifiedTime': '2019-07-12T20:00:01.793458+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC24RS_V3'}


### 2.2 Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [29]:
project_folder = './pytorch-pretrained-BERT'

Make a local clone of the original [PyTorch reimplementation](https://github.com/huggingface/pytorch-pretrained-BERT) repository

In [28]:
!git clone -b v0.4.0 https://github.com/huggingface/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
Note: checking out 'e1bfad48464a716fc65eac1f89a9f4c4915fd6db'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



### 2.3 Prepare your training script
Let us prepare the training script to run the fine-tuning script `run_squad.py`, that is already provided at [the original repository](https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_squad.py). Please refer [here](https://github.com/huggingface/pytorch-pretrained-BERT#fine-tuning-with-bert-running-the-examples) for more detail about the script. 

The original `run_squad.py` script uses PyTorch distributed launch untility to launch multiple processes across nodes and GPUs. [Azure-BERT](https://github.com/microsoft/AzureML-BERT) created by Microsoft prepared a modified version [bert_run_squad_azureml.py](https://github.com/microsoft/AzureML-BERT/blob/master/PyTorch/run_squad_azureml.py) so that we can launch it based on AzureML build-in MPI backend.

Let's retrieve and copy the training script [bert_run_squad_azureml.py](.\bert_run_squad_azureml.py), evaluation script for SQuAD v1.1 [evaluate-v1.1.py](https://github.com/allenai/bi-att-flow/blob/498c8026d92a8bcf0286e2d216d092d444d02d76/squad/evaluate-v1.1.py) and the helper utility script for Horovod [azureml_bert_util.py](https://github.com/microsoft/AzureML-BERT/blob/master/PyTorch/azureml_bert_util.py) into our project directory.

In [7]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/allenai/bi-att-flow/498c8026d92a8bcf0286e2d216d092d444d02d76/squad/evaluate-v1.1.py', filename= project_folder + '\\evaluate_squad.py')
urllib.request.urlretrieve('https://raw.githubusercontent.com/microsoft/AzureML-BERT/master/finetune/PyTorch/azureml_bert_util.py', filename= project_folder + '\\azureml_bert_util.py')

('./pytorch-pretrained-BERT\\evaluate_squad.py',
 <http.client.HTTPMessage at 0x25103433c88>)

In [61]:
shutil.copy('bert_run_squad_azureml.py', project_folder)

'./pytorch-pretrained-BERT\\bert_run_squad_azureml.py'

### 2.4 Create a PyTorch estimator for fine-tuning
Let us create a new PyTorch estimator to run the fine-tuning script `run_squad_azureml.py`. To use AML's tracking and metrics capabilities, we need to add a small amount of AzureML code inside the training script.

In `run_squad_azureml.py`, we will log some metrics to our AML run. To do so, we will access the AML run object within the script:
```Python
from azureml.core.run import Run
run = Run.get_context()
```
Further within `run_squad_azureml.py`, we log learning rate, training loss and prediction scores the model achieves as:
```Python
run.log('lr', np.float(args.learning_rate))
...

for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")): 
    ...
    run.log('train_loss', np.float(loss))

..
```
These run metrics will become particularly important when we begin hyperparameter tuning our model in the "Tune model hyperparameters" section below.

Then, AzureML PyTorch estimator can be defined as below. We use `azuremlsamples/bert:torch-1.0.0-apex-cuda9` as the base docker image with [dockerfile](./dockerfile). In this example, we use STANDARD_NC24rs_v3 which has 4 GPUs. Thus, we can set `process_count_per_node=4`.

In [8]:
mpiConfig=MpiConfiguration()
mpiConfig.process_count_per_node=4

estimator = PyTorch(source_directory=project_folder,
                    compute_target=gpu_compute_target,
                    script_params = {
                          '--bert_model':'bert-large-uncased',
                          '--do_train' : '',
                          '--do_predict': '',
                          '--train_file': ds.path('squad/train-v1.1.json').as_mount(),
                          '--predict_file': ds.path('squad/dev-v1.1.json').as_mount(),
                          '--max_seq_length': 384,
                          '--train_batch_size': 8,
                          '--learning_rate': 6.8e-5,
                          '--num_train_epochs': 2.0,
                          '--doc_stride': 128,
                          '--seed': 32,
                          '--init_gradient_accumulation_steps':2,
                          '--target_gradient_accumulation_steps':16,
                          '--accumulation_warmup_proportion':0.25,
                          '--output_dir': './outputs',
                          '--loss_scale':256,
                    },
                    custom_docker_image='azuremlsamples/bert:torch-1.0.0-apex-cuda9',
                    entry_script='bert_run_squad_azureml.py',
                    node_count=2,
                    distributed_training=mpiConfig,
                    framework_version='1.1',
                    use_gpu=True)
estimator._estimator_config.environment.python.user_managed_dependencies=True

**Note: You can try with `--bert_model':'bert-base-uncased`to run a smaller bert model faster.**

### 2.5 Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [9]:
experiment_name = 'BERT-SQuAD'
experiment = Experiment(ws, name=experiment_name)

### 2.6 Submit and Monitor your run
AzureML can automatically create figures on the loss and time, which is eaiser to track the performance as in the following figure shown the train loss v.s. the number of iterations:
![train_loss_bert](https://nlpbp.blob.core.windows.net/images/train_loss_bert.PNG)

The Jupyter widget would be like this:
![train_loss_bert](https://nlpbp.blob.core.windows.net/images/bert_widget.PNG)

In [74]:
run = experiment.submit(estimator)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

**Cancel the job**

It's better to cancel the job manually to make sure you do not waste resources.

 ```python
# Cancel the job with id.
job_id = "BERT-SQuAD_1562612876_bab5b3af"
run = get_run(experiment, job_id)

# Cancel jobs.
run.cancel()
 ```

To achieve over **90.5 F1 score** and **83.5 Exact-Match** with `SQuAD v1.1` dataset, it requires **2** epochs when fine-tune with `BERT large` model. Below please find the elapsed time using deferent Azure GPU VMs and configures. 

The default configuration in this notebook uses 2 `STANDARD_NC24rs_v3` (8 x V100) with `fp16` enabled. The training phase should take **22 mins** to complete 2 epochs. 

|  GPU counts 	|    1 GPU    	|         2 GPU 	| 4 GPU      	| 8 GPU      	|
|------------:	|:-----------:	|--------------:	|------------	|------------	|
| NCv3-series 	|     340 mins  |    180 mins 	    |    80 mins 	|   48 mins 	|
| NCv3 with fp16|     140 mins  |    79 mins 	    |    38 mins 	|   22 mins 	|

The performance with different VMs with `fp16` enabled (Duration = training time + preparing time):

|  VM Size 	|  GPU counts|    Node counts|    Duration    	|         F1 	| EM     	| Pretrained BERT      	|
|------------:	|:-----------:	|--------------:	|------------	|------------	|------------	|------------	|
| NC6sv3 |   4 |  4 |  31 mins  |    88.24 	    |    80.59 	|   Base 	|
| NC6sv3 |   4 |  4 |  80 mins  |    90.78 	    |    83.96 	|   Large 	|
| NC24rsv3 |  4 |   1 |  19 mins  |    86.18 	    |    77.90 	|   Base 	|
| NC24rsv3 |  4 |   1 |  46 mins  |    90.53 	    |    83.56 	|   Large 	|
| NC24rsv3 |  8 |   2 |  19 mins  |    87.47 	    |    79.52 	|   Base 	|
| NC24rsv3 |  8 |   2 |  32 mins  |    90.57 	    |    83.58 	|   Large 	|

## 3 Fine-Tuning BERT with Hyperparameter Tuning

We would also like to optimize our hyperparameter, `learning rate`, using Azure Machine Learning's hyperparameter tuning capabilities.

### 3.1 Start a hyperparameter sweep
First, we will define the hyperparameter space to sweep over. In this example we will use random sampling to try different configuration sets of hyperparameter to minimize our primary metric, the f1 score (`f1`). For simplicity, we tune the BERT base model with  `--bert_model':'bert-base-uncased` and  `node_count=1`.

In [10]:
param_sampling = RandomParameterSampling( {
         'learning_rate': uniform(5e-5, 9e-5),
    }
)
hyperdrive_config = HyperDriveConfig(estimator=estimator,
                                         hyperparameter_sampling=param_sampling, 
                                         primary_metric_name='f1',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=8,
                                         max_concurrent_runs=4)

Finally, lauch the hyperparameter tuning job.

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

### 3.2 Monitor HyperDrive runs
We can monitor the progress of the runs with the following Jupyter widget. 
![](https://nlpbp.blob.core.windows.net/images/bert_tune.PNG)
![](https://nlpbp.blob.core.windows.net/images/bert_tune2.PNG)

In [ ]:
RunDetails(hyperdrive_run).show()

You can see the experiment progress from this notebook by using `azureml.widgets.RunDetails(hd_run).show()` or check from the Azure portal with the url link you can get by running `hd_run.get_portal_url()`.
To load an existing Hyperdrive run, use `hd_run = hd.HyperDriveRun(exp, <user-run-id>, hyperdrive_run_config=hd_run_config)`. You also can cancel a run with `hd_run.cancel()`.

**Cancel the hyper drive run to save the resources**
 ```python
# Cancel the hyper drive
hyperdrive_run.cancel()
 ```

### 3.3 Find and register the best model
Once all the runs complete, we can find the run that produced the model with the highest F1 score. The F1 score with default learning rate is **86.18** in [Submit and Monitor your run](#2.6-Submit-and-Monitor-your-run) . The best F1 score is **87.01** after tuning with `learning rate=0.000090`.

In [20]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best Run is:\n  F1 score: %.2f \n  Learning rate: %f' % (best_run_metrics['f1'], best_run_metrics['lr']))

Run(Experiment: BERT-SQuAD,
Id: BERT-SQuAD_1562966635446_2,
Type: azureml.scriptrun,
Status: Completed)
Best Run is:
  F1 score: 87.01 
  Learning rate: 0.000090


## References

1. Devlin, Jacob and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina, [*BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding*](https://arxiv.org/abs/1810.04805), ACL, 2018.
2. Pranav Rajpurkar, Jian Zhang, Konstantin Lopyrev, Percy Liang, [*SQuAD: 100,000+ Questions for Machine Comprehension of Text*](https://arxiv.org/abs/1606.05250), EMNLP, 2016. Dataset available at [https://rajpurkar.github.io/SQuAD-explorer/](https://rajpurkar.github.io/SQuAD-explorer/).